In [2]:
from tkinter import filedialog
import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np
import os, copy
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
# from spec_to_color import spec_to_color

In [3]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [17]:
#load data

def open_pkl(fname):
    with open(fname, 'rb') as f:
        data = pkl.load(f)
    return data

filelist = filedialog.askopenfilenames()

data_list = []
stats_list = []

for data_file in filelist:
    d = open_pkl(data_file)
    d2 = copy.deepcopy(d)

    if d2['PL'] != None:
        del d2['PL']['energy'], d2['PL']['photons'], d2['PL']['freq_spectrum'], d2['PL']['gain_spectrum']
    if d2['absorption'] != None:
        del d2['absorption']['absorbance'], d2['absorption']['transmittance'], d2['absorption']['reference'], d2['absorption']['sample']
    #del d2['TE']['raw_data'], d2['TE']['fitted_data']
    data_list.append(d)
    stats_list.append(d2)
    print(data_file)

folder = os.path.dirname(filelist[0])
folder




C:/Users/smily/Dropbox/PythonScript/HPLCMS_characterization/sample_measured/20210129/3_c2_evap_3_3.725min_v10.pkl


'C:/Users/smily/Dropbox/PythonScript/HPLCMS_characterization/sample_measured/20210129'

In [ ]:
#create pandas data flame

df = pd.io.json.json_normalize(stats_list)

In [25]:
print(data_list[0].keys())
print(data_list[0]['PL'].keys())
print(data_list[0]['uv'].keys())
#print(data_list[0]['TE'].keys())
print(data_list[0]['metadata'].keys())
print(data_list[0]['job'].keys())
print(data_list[0]['TE'].keys())
#print(data_list[0]['metadata']['sample'].keys())

dict_keys(['uv', 'absorption', 'PL', 'TE', 'metadata', 'job'])
dict_keys(['energy', 'photons', 'freq_spectrum', 'gain_spectrum', 'relative_QY', 'max', 'lambda_max', 'time_trace', 'int_time_trace', 'maintenance', 'degradation_rate', 'maintenance(at_1min)', 'exposure', 'max_gain_factor', 'max_gain_wavelength'])
dict_keys(['reference', 'absorption', 'absorbance_time_trace', 'absorbance_maintenance', 'degradation_rate', 'absorbance_maintenance(at_1min)', 'wavelength'])
dict_keys(['absorption', 'PL', 'TE', 'dilution', 'redissolution', 'data_processing', 'data_path'])
dict_keys(['injection_name', 'target_name', 'retention_time', 'vial_number', 'average_absorbance_peak', 'average_absorbance_375', 'sample_volume', 'target_smiles', 'filename'])
dict_keys(['raw_data', 'metadata', 'fitted_data', 'fitting_results'])


In [ ]:
df.head()

In [ ]:
#output csv

fname = '%s/df.csv' %folder
df.to_csv(fname)

In [18]:
#calculation range
calc_range = [300, 800]

l_index = np.where(data_list[0]['PL']['energy'][0] >= calc_range[0])[0][0]
u_index = np.where(data_list[0]['PL']['energy'][0] >= calc_range[1])[0][0]
print(l_index, u_index)
print(data_list[0]['PL']['energy'][0][l_index], data_list[0]['PL']['energy'][0][u_index] )

131 787
300.1890812457546 800.0229004530321


In [6]:
#plot legend
conc_list = df['metadata.sample.concentration(uM)'].values
l = []
concentrations = [c for c in conc_list if c not in l and not l.append(c)]  

text = "concentration(uM)\n"
for c in concentrations:
    text += '%s\n' %c 


NameError: name 'df' is not defined

In [13]:
SC = spec_to_color.Spec_to_Color()

for data in data_list:
    rgb = SC.spec_to_rgb(data['PL']['energy'])
    #print(rgb)
    print('%s : %s' %(data['job']['target_name'], SC.hex_to_str(rgb)))
    

8 : #08ff90
12 : #00ffa7
13 : #ff4200
14 : #ff2515
17 : #005bff
18 : #0039ff
20 : #00fff8
21 : #a4ff0a
22 : #00b9ff
23 : #0cff60
24 : #008fff
25 : #00ff77
26 : #ffbc00
28 : #00ffaf
30 : #00ff7c
31 : #0cff61
32 : #0094ff
33 : #00ffe8
35 : #06ffa5
36 : #adff05
37 : #00ffa8
39 : #0062ff
40 : #00aeff
42 : #004aff
46 : #00abff


In [5]:
label = [0,5,10,20,30,40]
label = [1,2]

text = "bubbling time(min)\n"
for l in label:
    text += '%s\n' %l


In [14]:
#plot PL emission
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1, xlabel = 'wavelength / nm', ylabel = 'intensity /(energy/nm/s)', xlim = (300, 800))
for d in data_list:
    ax1.plot(d['PL']['energy'][0], d['PL']['energy'][1])
ax1.text(0.9, 0.9, text, ha='right',  va='top', transform = ax1.transAxes)
#plt.savefig('%s/PL_spectra.png' %folder)
plt.show()



In [16]:
#plot absorption spectrum
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1, xlabel = 'wavelength / nm', ylabel = 'absorbance', xlim = (300, 800), ylim = (-0.005, 0.1))
for d in data_list:
    ax1.plot(d['absorption']['absorbance'][0], d['absorption']['absorbance'][1], label = 'a')
ax1.text(0.9, 0.9, text, ha='right',  va='top', transform = ax1.transAxes)
#plt.savefig('%s/absorption_spectra.png' %folder)
plt.legend()
plt.show()

In [16]:
#plot PL and absorption spectrum
fig = plt.figure()
plt.rcParams.update({'font.size': 15})
ax1 = fig.add_subplot(1,1,1, xlabel = 'wavelength / nm', ylabel = 'Normalized abs. and PL intensity', xlim = (250, 700), ylim = [-0.1, 1.1])
conc = []
linewidth = 6
for d in data_list:
#     if d['metadata']['sample']['concentration(uM)'] not in conc:
    # if d['metadata']['sample']['concentration(uM)'] not in conc and d['metadata']['sample']['concentration(uM)'] == 10:
    ax1.plot(d['absorption']['absorbance'][0], d['absorption']['absorbance'][1]/np.max(d['absorption']['absorbance'][1][l_index:u_index+1]),\
            linewidth = linewidth)
    ax1.plot(d['PL']['energy'][0], d['PL']['energy'][1]/np.max(d['PL']['energy'][1][l_index:u_index+1]), linewidth = linewidth)
#         conc.append(d['metadata']['sample']['concentration(uM)'] )
#ax1.text(0.9, 0.9, text, ha='right',  va='top', transform = ax1.transAxes)

#plt.savefig('%s/abs_PL_20uM.png' %folder)
plt.show()

In [ ]:
#PL time trace
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1, xlabel = 'time/s', ylabel = 'Normalized PL intensity')
for i, d in enumerate(data_list):
    ax1.plot(d['PL']['peak_time_trace'][0], d['PL']['peak_time_trace'][1]/d['PL']['peak_time_trace'][1][0],label = label[i])
    # ax1.plot(d['PL']['peak_time_trace'][0], d['PL']['peak_time_trace'][1]/d['PL']['peak_time_trace'][1][0],label = d['metadata']['sample']['concentration(uM)'])
# ax1.text(0.9, 0.9, text, ha='right',  va='top', transform = ax1.transAxes)
ax1.legend(loc = 'upper left', ncol = 1, bbox_to_anchor=(1, 1))
#plt.savefig('%s/absorption_spectra.png' %folder)
plt.show()

In [ ]:
#uv absorbance time trace
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1, xlabel = 'time/s', ylabel = 'Normalized absorbance')
for i, d in enumerate(data_list):
    ax1.plot(d['uv']['absorbance_time_trace'][0], d['uv']['absorbance_time_trace'][1]/d['uv']['absorbance_time_trace'][1][0], label = label[i])
    # ax1.plot(d['uv']['absorbance_time_trace'][0], d['uv']['absorbance_time_trace'][1]/d['uv']['absorbance_time_trace'][1][0], label = d['metadata']['sample']['concentration(uM)'])
#ax1.text(0.9, 0.9, text, ha='right',  va='top', transform = ax1.transAxes)
#plt.savefig('%s/absorption_spectra.png' %folder)
ax1.legend(loc = 'upper left', ncol = 1, bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
#uv absorption time trace
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1, xlabel = 'time/s', ylabel = 'Normalized absorbed energy')
for d in data_list:
    a  = 1 - 10**(-d['uv']['absorbance_time_trace'][1])
    ax1.plot(d['uv']['absorbance_time_trace'][0], a/a[0])
ax1.text(0.9, 0.9, text, ha='right',  va='top', transform = ax1.transAxes)
#plt.savefig('%s/absorption_spectra.png' %folder)
plt.show()

In [27]:
#plot PL transient
fig = plt.figure()
plt.rcParams.update({'font.size': 15})
linewidth = 6
ax1 = fig.add_subplot(1,1,1, xlabel = 'time/ns', ylabel = 'photon count', xlim = (10, 30), ylim = (1, 100000))
for d in data_list:
    ax1.plot(d['TE']['raw_data'][0], d['TE']['raw_data'][1], linewidth = linewidth) 
    ax1.plot(d['TE']['fitted_data'][0], d['TE']['fitted_data'][1], linewidth = linewidth)
ax1.set_yscale('log')
    #ax1.plot(d['TE']['fitted_data'][0], d['TE']['fitted_data'][1])
#ax1.text(0.9, 0.9, text, ha='right',  va='top', transform = ax1.transAxes)
plt.savefig('%s/PL_lifetime.png' %folder)
plt.show()

In [ ]:
def conc_plot(df, data, yrange = None, ylabel = None, fitting = False, filename = None):
    d_mean = []
    d_std = []
    for conc in concentrations:
        d = df[df['metadata.sample.concentration(uM)'] == conc][data]
        d_mean.append(d.mean())
        d_std.append(d.std())
    error = np.asarray(d_std) *3
    plt.errorbar(concentrations, d_mean, yerr = error, capsize=5, fmt='o', markersize=0, ecolor='k')
    plt.scatter(concentrations, d_mean)
    plt.xlabel("concentration/uM")
    if yrange:
        plt.ylim(yrange)
    if ylabel:
        plt.ylabel(ylabel)
    if fitting == True:
        x = [[c] for c in df['metadata.sample.concentration(uM)']]
        reg=LinearRegression(fit_intercept = False).fit(x, df[data])
        y_pred = [reg.intercept_ + reg.coef_[0] * conc for conc in df['metadata.sample.concentration(uM)'].values]
        r2 = r2_score(df[data], y_pred)
        plt.plot(df['metadata.sample.concentration(uM)'], y_pred, linestyle = 'dashed', color = 'k')
        plt.text(0.2, max(df[data])*0.8, 'R2 = {:.4f}\ny = {:.3e}x'.format(r2, reg.coef_[0]))
    if filename:
        plt.savefig(filename)

    plt.show()  

In [ ]:
fname = '%s/conc-abs.png' %folder
conc_plot(df, data = 'PL.relative_QY', yrange = (-0.02, 0.7), ylabel = 'absorbance(lambda_max)', fitting = False, filename = fname)

In [ ]:
d_list = ['PL.max_gain_factor', 'PL.relative_QY', 'TE.fitting_results.tau']
labels = ['max gain factor\n(1E-24 cm2 s)', 'relative_PLQY*\n(ref: DPA)', '1/PL lifetime (1E9 s-1)']
d_mean = [[] for _ in range(len(concentrations))]
d_std = [[] for _ in range(len(concentrations))]

for i, conc in enumerate(concentrations):
    for j, data in enumerate(d_list):
        d = df[df['metadata.sample.concentration(uM)'] == conc][data]
        if j == 0:
            d = d * 1E24
        elif j == 1:
            d = d/0.53
        elif j == 2:
            d = 1/d
        d_mean[i].append(d.mean())
        d_std[i].append(d.std()*3)

x = np.arange(len(d_list))
bar_width = 0.8/len(concentrations)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
for i, conc in enumerate(concentrations):
    ax.bar(x + i * bar_width, d_mean[i], yerr = d_std[i], width = bar_width, label = '%s uM' %conc, ecolor="black", capsize=3)
ax.set_ylim(0, 1.2)
ax.set_xticks(x + bar_width * round(len(concentrations)/2))
ax.set_xticklabels(labels)
ax.legend(loc = 'upper right', ncol = 1, bbox_to_anchor=(1, 1))
ax.grid(which = "major", axis = "y", color = "k", alpha = 0.3,
        linestyle = "--", linewidth = 1)
plt.savefig('%s/objectives.png' %folder)
plt.show()